<a href="https://colab.research.google.com/github/IvlevaAnna1/Hometasks/blob/main/sql_1_HW_IvlevaAnna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
con = sqlite3.connect(':memory:')

In [5]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [10]:
for table in table_names:
    data = pd.read_csv(f'sample_data/{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [12]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [14]:
pd.read_sql(
    '''
    SELECT c.CustomerName, c.Country, c.Address FROM Customers c
    where c.Country in ('Germany', 'France') or c.City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [21]:
pd.read_sql(
    '''
    SELECT c.Country, count(c.Country) FROM Customers c
    group by c.Country
    order by count(c.Country) DESC LIMIT 3
    ''',
    con,
)

,Country,count(c.Country)
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [42]:
pd.read_sql(
    '''
    SELECT * FROM (SELECT s.ShipperName, o.OrderDate, ROW_NUMBER() OVER (ORDER BY o.OrderDate ASC) AS rownumber
    FROM Orders o
    left join Shippers s on o.ShipperID = s.ShipperID) t1
    where t1.rownumber = 10
    ''',
    con,
)

,ShipperName,OrderDate,rownumber
0,Federal Shipping,02.10.1996,10


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [88]:
pd.read_sql(
    '''
    select pr.ProductName,pr.Price from OrderDetails o
    left join Products pr on o.ProductID = pr.ProductID
    where o.OrderID = (
    SELECT OrderID FROM
    (SELECT *, sum(od.Quantity*p.Price ) over(partition by OrderID) as MASORDERSUM
    FROM OrderDetails od
    left join Products p on od.ProductID = p.ProductID
    order by MASORDERSUM DESC limit 1) t)

    ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,81.0
1,Côte de Blaye,263.5
2,Camembert Pierrot,34.0
3,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [108]:
pd.read_sql(
    '''
    select o.OrderID, od.Quantity, pr.ProductName from Orders o
    left join OrderDetails od on o.OrderID = od.OrderID
    left join Products pr on od.ProductID = pr.ProductID
    where od.ProductID in (
    select ProductID
    FROM (
    SELECT *, sum(od.Quantity) OVER(PARTITION BY od.ProductID) as SUMPRODUCTQUANTITY
    FROM OrderDetails od
    left join Products p on od.ProductID = p.ProductID
    order by SUMPRODUCTQUANTITY DESC
    limit 1) t)
    ''',
    con,
)

,OrderID,Quantity,ProductName
0,10253,20,Gorgonzola Telino
1,10272,40,Gorgonzola Telino
2,10273,15,Gorgonzola Telino
3,10325,4,Gorgonzola Telino
4,10335,25,Gorgonzola Telino
5,10342,56,Gorgonzola Telino
6,10356,30,Gorgonzola Telino
7,10359,70,Gorgonzola Telino
8,10363,20,Gorgonzola Telino
9,10374,30,Gorgonzola Telino


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [107]:
pd.read_sql(
    '''
    SELECT *
    FROM OrderDetails od
    where ProductID = 31
    ''',
    con,
)

,OrderDetailID,OrderID,ProductID,Quantity
0,15,10253,31,20
1,64,10272,31,40
2,67,10273,31,15
3,205,10325,31,4
4,233,10335,31,25
5,253,10342,31,56
6,287,10356,31,30
7,297,10359,31,70
8,309,10363,31,20
9,337,10374,31,30


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)